# Geospatial BI & Data Viz Project - Part 2

- VISUALIZATION PROJECT Geospatial Business Intelligence (BI)
    * Make a geospatial analysis of the `companies` dataset
    * Things you know:
        - You have a software company with 50 employees
        - The company creates video games
        - Roles in your company: 20 developers, 20 Designers/Creatives/UX/UI and 10 executives/managers
    * Do an analysis about placing the new company offices in the best environment based on the following criteria:
        - There should be software engineers working around
        - The surroundings must have a good ratio of big companies vs startups
        - Ensure you have in your surroundings companies that cover the interests of your team
        - Avoid old companies, prefer recently created ones

In [1]:
# import pymongo to connect Python with MongoDB
from pymongo import MongoClient
# to work with stats
import pandas as pd
# to work with dataframes
import numpy as np
# to work with json
from pandas.io.json import json_normalize

### 1. Prepare the data: extract from the companies database only the relevant information for the challenge.

* for geolocation analysis & geospatial visualization
    - name
    - offices.country_code, offices.city, offices.latitude, offices.longitude 
        
* to determine if the company is old o recent
    - founded_year --> company size

* to define if it's a small (startup) or a big company:
    - number_of_employees --> company size
    - investments.funding_round.round_code: 'Angel','seed' --> to associate to startup category
    - investments.funding_round.funded_year
    - ipo.pub_year,ipo.valuation_amount --> to associate with big company
    https://support.crunchbase.com/hc/en-us/articles/115010458467-Glossary-of-Funding-Types

* to match with our team interests: technology & videogames
    - category_code: 'software', 'web', 'games_video' --> to filter as 'best match' for our team
    - description: 'software','technology', 'Platform','Social network' --> for a qualitative analysis
    - tag_list: 'network', 'online-communities','projects', etc --> for a qualitative analysis

In [2]:
# connecting on default host and port
client = MongoClient ('localhost', 27017)

# loading the database
db = client['companies']

# getting the collection
companies = db['companies']

# defining query to get the relevant information:
query = db.companies.find({'founded_year': {'$gt': year }},{'name':1, 'founded_year': 1, '_id': 0, 
'number_of_employees':1,'offices.latitude':1, 'offices.longitude':1, 'funding_rounds.raised_amount':1, 
'funding_rounds.raised_currency_code':1}) 
data = pd.DataFrame(query)